In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os

In [51]:
def visualization(folder_name: str, dataframe: str, table: str, resolution: int, title_left: str, title_right: str) -> None:
    eval_df = pd.read_csv(dataframe)

    if not os.path.exists(folder_name):
        # Create the folder
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created.")
    else:
        print(f"Folder '{folder_name}' already exists.")

    averages = {}
    stan = {}
    median = {}

    for column in eval_df.columns:
        if eval_df[column].dtype in [int, float]:
            averages[column] = eval_df[column].mean()
            stan[column] = eval_df[column].std()
            median[column] = eval_df[column].median()
            
    with open(f"{folder_name}/{table}", "w") as file:
        file.write(" \\begin{table}[] \n ")
        file.write(" \\begin{tabular}{l|llll|llll} \n")
        file.write(" metric              & nDCG@5 & nDCG@10 & nDCG@20 & nDCG@50 & AWRF@5 & AWRF@10 & AWRF@20 & AWRF@50 \\\\ \hline  \n")
        file.write(f" average score      & {averages['nDCG5']:.2f} & {averages['nDCG10']:.2f} & {averages['nDCG20']:.2f} & {averages['nDCG50']:.2f} & {averages['awrf5']:.2f} & {averages['awrf10']:.2f} & {averages['awrf20']:.2f} & {averages['awrf50']:.2f} \\\\ \n ")
        file.write(f" standard deviation & {stan['nDCG5']:.2f}     & {stan['nDCG10']:.2f}     & {stan['nDCG20']:.2f}     & {stan['nDCG50']:.2f}     & {stan['awrf5']:.2f}     & {stan['awrf10']:.2f}     & {stan['awrf20']:.2f}     & {stan['awrf50']:.2f} \\\\ \n")
        file.write(f" median             & {median['nDCG5']:.2f}   & {median['nDCG10']:.2f}   & {median['nDCG20']:.2f}   & {median['nDCG50']:.2f}   & {median['awrf5']:.2f}   & {median['awrf10']:.2f}   & {median['awrf20']:.2f}   & {median['awrf50']:.2f} \n")
        file.write(" \\end{tabular} \n")
        file.write(" \\end{table} ")

    plt.figure(figsize=(5, 6), dpi=resolution)
    
    averages_slice2 = dict(list(averages.items())[:4])
    median_slice2 = dict(list(median.items())[:4])
    std_slice2 = dict(list(stan.items())[:4])
    
    plt.plot(list(averages_slice2.keys()), list(averages_slice2.values()), marker='o', color='skyblue', label='Averages', linewidth=2)
    plt.plot(list(median_slice2.keys()), list(median_slice2.values()), marker='o', color='orange', label='Median', linewidth=2)
    plt.plot(list(std_slice2.keys()), list(std_slice2.values()), marker='o', color='green', label='Standard deviation', linewidth=2)
    
    plt.title(title_left)
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1)  # Set y-axis limit to a maximum of 1
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'{folder_name}/{title_left}.png', dpi=resolution)
    plt.close()

    plt.figure(figsize=(5, 6), dpi=resolution)

    # Slice dictionaries to consider only columns 5 to 8
    averages_slice = dict(list(averages.items())[4:])
    median_slice = dict(list(median.items())[4:])
    std_slice = dict(list(stan.items())[4:])

    # Get keys and values for the dictionaries
    averages_keys, averages_values = list(averages_slice.keys()), list(averages_slice.values())
    median_keys, median_values = list(median_slice.keys()), list(median_slice.values())
    std_keys, std_values = list(std_slice.keys()), list(std_slice.values())

    # Plot averages as lines
    plt.plot(averages_keys, averages_values, marker='o', color='skyblue', label='Averages', linewidth=2)
    plt.plot(median_keys, median_values, marker='o', color='orange', label='Median', linewidth=2)
    plt.plot(std_keys, std_values, marker='o', color='green', label='Standard deviation', linewidth=2)

    plt.title(title_right)
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1)  # Set y-axis limit to a maximum of 1
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'{folder_name}/{title_right}.png', dpi=resolution)
    plt.close()
    
    # Open the two images to be merged
    image1 = Image.open(f"{folder_name}/{title_right}.png")
    image2 = Image.open(f"{folder_name}/{title_left}.png")

    # Check if the images have the same size
    if image1.size == image2.size:
        merged_image = Image.new("RGB", (image1.width * 2, image1.height))

        # Merge the images
        merged_image.paste(image2, (0, 0))
        merged_image.paste(image1, (image1.width, 0))

        # Save the merged image
        merged_image.save(f"{folder_name}/merged_image.png")
    else:
        print("Images have different sizes, cannot merge.")

In [52]:
visualization("images","BM25 results.csv","output.txt",100,"nDCG","AWRF")

Folder 'images' already exists.
